# Visualizations of Optimized Network Results

### Setting Up

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
import shapely
import folium
import geojson
import math
import osmnx as ox
from rtree import index as rtree_index
import pickle
import copy
import utm
import seaborn as sns
import matplotlib.pyplot as plt

from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, LineString, Point
from geopy.distance import geodesic
from shapely.ops import split

from __future__ import absolute_import, division
from math import radians, sin, cos, sqrt, atan2, exp, log
import webbrowser
import random
from scipy.spatial import KDTree
from scipy.spatial.distance import euclidean

ox.settings.log_console=True
ox.settings.use_cache=True

In [ ]:
# Defining classes for the dataframes      
class stopCandidate:
    def __init__(self, lat, long, isTranspo, id):
        self.lat = lat
        self.long = long
        self.isTranspo = isTranspo
        self.id = id #Stop ID
        
    def getLat(self):
        return self.lat
    
    def getLong(self):
        return self.long
    
    def getDegree(self):
        return self.degree
    
class networkObj:
    def __init__(self, route_network, graph):
        self.fitness_score = 0
        self.graph = graph
        self.route_network = route_network
        
class Route:
    def __init__(self, route, route_id, distance):
        self.route = route
        self.route_id = route_id
        self.distance = distance

### Functions

In [ ]:
# Import and Export networks or graphs to pickle
def export_networks(networks, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'wb') as f:
        pickle.dump(networks, f)

def import_networks(path):
    with open(path, 'rb') as f:
        routes = pickle.load(f)
    return routes

In [ ]:
# Import recent population backup file
def import_recent_population(city_name):
    if city_name == 'Manila':
        filename = f"GA Backup Files/GA_recent_population_Manila_backup.pikl"
    elif city_name == 'Makati':
        filename = f"GA Backup Files/GA_recent_population_Makati_backup.pikl"
    else:
        filename = f"GA Backup Files/GA_recent_population_backup.pikl"
        
    data = import_networks(filename)
    
    return data['population_num'], data['population']
    

# Import best population 
def import_recent_best_population(city_name, iteration=''):
    if city_name == 'Manila':
        filename = f"GA Backup Files/GA_recent_best_population_Manila_backup{iteration}.pikl"
    elif city_name == 'Makati':
        filename = f"GA Backup Files/GA_recent_best_population_Makati_backup{iteration}.pikl"
    else:
        filename = f"GA Backup Files/GA_recent_best_population_backup{iteration}.pikl"
    
    data = import_networks(filename)
    
    return data['population_num'], data['population']

# Import current best population into pikl in case of errors
def import_max_score_list(city_name):
    if city_name == 'Manila':
        filename = f"GA Backup Files/GA_max_score_list_Manila_backup.pikl"
        filename2 = f"GA Backup Files/GA_max_score_list_Manila_backup2.pikl"
    elif city_name == 'Makati':
        filename = f"GA Backup Files/GA_max_score_list_Makati_backup.pikl"
        filename2 = f"GA Backup Files/GA_max_score_list_Makati_backup2.pikl"
    else:
        filename = f"GA Backup Files/GA_max_score_list_backup.pikl"
        filename2 = f"GA Backup Files/GA_max_score_list_backup2.pikl"
    
    data = import_networks(filename)
    data2 = import_networks(filename2)

    return data['population_num'] + data2['population_num'], data['max_list'] + data2['max_list']

In [ ]:
# Longest Route
def get_longest_route(network_routes):
    sorted_routes = sorted(network_routes, key=lambda x: x.length, reverse=True)
    longest_route = sorted_routes[0]
    
    stop_list = [] # Get all the stops
    for connection in longest_route.route:
        if connection[0] not in stop_list:
            stop_list.append(connection[0])
        if connection[-1] not in stop_list:
            stop_list.append(connection[-1])
    
    return longest_route, stop_list

# Shortest Route
def get_shortest_route(network_routes):
    sorted_routes = sorted(network_routes, key=lambda x: x.length, reverse=True)
    shortest_route = sorted_routes[-1]
    
    stop_list = [] # Get all the stops
    for connection in shortest_route.route:
        if connection[0] not in stop_list:
            stop_list.append(connection[0])
        if connection[-1] not in stop_list:
            stop_list.append(connection[-1])
    
    return shortest_route, stop_list

# Average Route Length
def get_average_route_length(network_routes):
    sum_length = sum([n.length for n in network_routes])
    average_length = sum_length / len(network_routes)
    return average_length

def get_longest_path_lengths(snapped_road_network_graph):
    path_lengths = []
    for node in snapped_road_network_graph.nodes():
        lengths = nx.single_source_dijkstra_path_length(snapped_road_network_graph, node, weight='distance')
        list_lengths = [n for target_node, n in lengths.items()]
        path_lengths.append(max(list_lengths))
    return path_lengths

#Network Diameter
def get_network_diameter(network_graph):
    path_lengths = get_longest_path_lengths(network_graph) # Get the sum of all possible
    max_path_length = max(path_lengths)
    return max_path_length


def average_degree(graph):
    degrees = [degree for _, degree in graph.degree()]
    avg_degree = sum(degrees) / len(degrees)
    return avg_degree

def average_number_of_stops_per_route(network_routes):
    num_stops = []
    for route in network_routes:
        seen = set()
        for connection in route.route:
            if connection[0] not in seen:
                seen.add(connection[0])
            
            if connection[-1] not in seen:
                seen.add(connection[-1])
                
        num_stops.append(len(seen))
            
    avg_stops = sum(num_stops) / len(num_stops)
    return avg_stops

In [ ]:
# Node Centrality
def get_node_centrality(network_graph):
    degree_centrality = nx.degree_centrality(network_graph)
    betweenness_centrality = nx.betweenness_centrality(network_graph)
    closeness_centrality = nx.closeness_centrality(network_graph)
    
    return degree_centrality, betweenness_centrality, closeness_centrality

### Import Data

#### Max Score List

In [ ]:
# Max Score List - MANILA
generation_num, max_score_list_Manila = import_max_score_list('Manila')
i = 0
print(f"Last Generation Num: {generation_num}")
for max_score, components in max_score_list_Manila:
    print(f"Generation {i}")
    print("Max Score: ", max_score)
    print(f"Weighted Connectivity: {components['Connectivity']}")
    print(f"- Transpo Stop Ratio: {components['Transpo Stop Ratio']}")
    print(f"- Num Intersections: {components['Num Intersections']}")
    print(f"- Average Transpo Stop Degree: {components['Average Transpo Degree']}")
    print(f"Weighted Network Coverage: {components['Network Coverage']}")
    print(f"Weighted Random Failure Score: {components['Random Failure Score']}")
    print(f"Weighted Target Failure Score: {components['Target Failure Score']}")
    print()
    i += 1

In [ ]:
# Max Score List - MAKATI
generation_num, max_score_list_Makati = import_max_score_list('Makati')
i = 0
print(f"Last Generation Num: {generation_num}")
for max_score, components in max_score_list_Makati:
    print(f"Generation {i}")
    print("Max Score: ", max_score)
    print(f"Weighted Connectivity: {components['Connectivity']}")
    print(f"- Transpo Stop Ratio: {components['Transpo Stop Ratio']}")
    print(f"- Num Intersections: {components['Num Intersections']}")
    print(f"- Average Transpo Stop Degree: {components['Average Transpo Degree']}")
    print(f"Weighted Network Coverage: {components['Network Coverage']}")
    print(f"Weighted Random Failure Score: {components['Random Failure Score']}")
    print(f"Weighted Target Failure Score: {components['Target Failure Score']}")
    print()
    i += 1

In [ ]:
# Max Score List - MANDALUYONG
generation_num, max_score_list_Mandaluyong = import_max_score_list('Mandaluyong')
i = 0
print(f"Last Generation Num: {generation_num}")
for max_score, components in max_score_list_Mandaluyong:
    print(f"Generation {i}")
    print("Max Score: ", max_score)
    print(f"Weighted Connectivity: {components['Connectivity']}")
    print(f"- Transpo Stop Ratio: {components['Transpo Stop Ratio']}")
    print(f"- Num Intersections: {components['Num Intersections']}")
    print(f"- Average Transpo Stop Degree: {components['Average Transpo Degree']}")
    print(f"Weighted Network Coverage: {components['Network Coverage']}")
    print(f"Weighted Random Failure Score: {components['Random Failure Score']}")
    print(f"Weighted Target Failure Score: {components['Target Failure Score']}")
    print()
    i += 1

#### Final Population

In [ ]:
generation_num, population_Manila = import_recent_best_population('Manila', '2')
generation_num, population_Makati = import_recent_best_population('Makati', '2')
generation_num, population_Mandaluyong = import_recent_best_population('Mandaluyong', '2')

#### Generation 0 Population

In [ ]:
# File Paths
Manila_pikl_filepath = "Saved Networks/Manila/"
Manila_map_filepath = "Saved Maps/Manila/"

Makati_pikl_filepath = "Saved Networks/Makati/"
Makati_map_filepath = "Saved Maps/Makati/"

Mandaluyong_pikl_filepath = "Saved Networks/Mandaluyong/"
Mandaluyong_map_filepath = "Saved Maps/Mandaluyong/"

In [ ]:
# Importing First Generation best network (Only to test the same routes) ONLY MIXED NETWORKS
Manila_generation0 = import_networks(f"{Manila_pikl_filepath}Manila_best_generation0.pkl")
Makati_generation0 = import_networks(f"{Makati_pikl_filepath}Makati_best_generation0.pkl")
Mandaluyong_generation0 = import_networks(f"{Mandaluyong_pikl_filepath}Mandaluyong_best_generation0.pkl")

### Fitness Score Graphs

#### Max Score VS Generations

In [ ]:
# Extract max_score and generations
max_scores_Manila = [item[0] for item in max_score_list_Manila]
max_scores_Makati = [item[0] for item in max_score_list_Makati]
max_scores_Mandaluyong = [item[0] for item in max_score_list_Mandaluyong]

max_length = max(len(max_scores_Manila), len(max_scores_Makati), len(max_scores_Mandaluyong))

if len(max_scores_Manila) < max_length:
    max_scores_Manila.extend([max_scores_Manila[-1]] * (max_length - len(max_scores_Manila)))
if len(max_scores_Makati) < max_length:
    max_scores_Makati.extend([max_scores_Makati[-1]] * (max_length - len(max_scores_Makati)))
if len(max_scores_Mandaluyong) < max_length:
    max_scores_Mandaluyong.extend([max_scores_Mandaluyong[-1]] * (max_length - len(max_scores_Mandaluyong)))

generations = list(range(1, max_length + 1))

# Create DataFrames for each location
data_Manila = pd.DataFrame({
    'Generation': generations,
    'Fitness Score': max_scores_Manila,
    'Location': 'Manila'
})

data_Makati = pd.DataFrame({
    'Generation': generations,
    'Fitness Score': max_scores_Makati,
    'Location': 'Makati'
})

data_Mandaluyong = pd.DataFrame({
    'Generation': generations,
    'Fitness Score': max_scores_Mandaluyong,
    'Location': 'Mandaluyong'
})

data = pd.concat([data_Manila, data_Makati, data_Mandaluyong])

# Plotting the graph
sns.lineplot(data=data, x='Generation', y='Fitness Score', hue='Location')
plt.xlabel('Generation')
plt.ylabel('Fitness Score')
plt.title('Fitness Score vs Generations for Each City')
plt.legend(title='Location')
plt.show()

#### Connectivity VS Generations

In [ ]:
# Extract max_score and generations
Connectivity_scores_Manila = [item[1]['Connectivity'] for item in max_score_list_Manila]
Connectivity_scores_Makati = [item[1]['Connectivity'] for item in max_score_list_Makati]
Connectivity_scores_Mandaluyong = [item[1]['Connectivity'] for item in max_score_list_Mandaluyong]

max_length = max(len(Connectivity_scores_Manila), len(Connectivity_scores_Makati), len(Connectivity_scores_Mandaluyong))

if len(Connectivity_scores_Manila) < max_length:
    Connectivity_scores_Manila.extend([Connectivity_scores_Manila[-1]] * (max_length - len(Connectivity_scores_Manila)))
if len(Connectivity_scores_Makati) < max_length:
    Connectivity_scores_Makati.extend([Connectivity_scores_Makati[-1]] * (max_length - len(Connectivity_scores_Makati)))
if len(Connectivity_scores_Mandaluyong) < max_length:
    Connectivity_scores_Mandaluyong.extend([Connectivity_scores_Mandaluyong[-1]] * (max_length - len(Connectivity_scores_Mandaluyong)))

generations = list(range(1, max_length + 1))

# Create a DataFrame for seaborn
import pandas as pd
data_Manila = pd.DataFrame({
    'Generation': generations,
    'Connectivity': Connectivity_scores_Manila,
    'Location': 'Manila'
})

data_Makati = pd.DataFrame({
    'Generation': generations,
    'Connectivity': Connectivity_scores_Makati,
    'Location': 'Makati'
})

data_Mandaluyong = pd.DataFrame({
    'Generation': generations,
    'Connectivity': Connectivity_scores_Mandaluyong,
    'Location': 'Mandaluyong'
})

# Combine all DataFrames into one
data = pd.concat([data_Manila, data_Makati, data_Mandaluyong])

# Plotting the graph
sns.lineplot(data=data, x='Generation', y='Connectivity', hue='Location')
plt.xlabel('Generation')
plt.ylabel('Connectivity Score')
plt.title('Connectivity vs Generations for Each City')
plt.legend(title='Location')
plt.show()

#### Coverage VS Generations

In [ ]:
# Extract max_score and generations
Network_coverage_Manila = [item[1]['Network Coverage'] for item in max_score_list_Manila]
Network_coverage_Makati = [item[1]['Network Coverage'] for item in max_score_list_Makati]
Network_coverage_Mandaluyong = [item[1]['Network Coverage']  for item in max_score_list_Mandaluyong]

max_length = max(len(Network_coverage_Manila), len(Network_coverage_Makati), len(Network_coverage_Mandaluyong))

if len(Network_coverage_Manila) < max_length:
    Network_coverage_Manila.extend([Network_coverage_Manila[-1]] * (max_length - len(Network_coverage_Manila)))
if len(Network_coverage_Makati) < max_length:
    Network_coverage_Makati.extend([Network_coverage_Makati[-1]] * (max_length - len(Network_coverage_Makati)))
if len(Network_coverage_Mandaluyong) < max_length:
    Network_coverage_Mandaluyong.extend([Network_coverage_Mandaluyong[-1]] * (max_length - len(Network_coverage_Mandaluyong)))

generations = list(range(1, max_length + 1))

# Create a DataFrame for seaborn
import pandas as pd
data_Manila = pd.DataFrame({
    'Generation': generations,
    'Network Coverage': Network_coverage_Manila,
    'Location': 'Manila'
})

data_Makati = pd.DataFrame({
    'Generation': generations,
    'Network Coverage': Network_coverage_Makati,
    'Location': 'Makati'
})

data_Mandaluyong = pd.DataFrame({
    'Generation': generations,
    'Network Coverage': Network_coverage_Mandaluyong,
    'Location': 'Mandaluyong'
})

# Combine all DataFrames into one
data = pd.concat([data_Manila, data_Makati, data_Mandaluyong])

# Plotting the graph
sns.lineplot(data=data, x='Generation', y='Network Coverage', hue='Location')
plt.xlabel('Generation')
plt.ylabel('Network Coverage Score')
plt.title('Network Coverage vs Generations for Each City')
plt.legend(title='Location', bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()

#### Vulnerability VS Generation

In [ ]:
# Extract max_score and generations
Vulnerability_scores_Manila = [item[1]['Random Failure Score'] for item in max_score_list_Manila]

Vulnerability_scores_Manila = Vulnerability_scores_Manila[:5000]
max_length = len(Vulnerability_scores_Manila)

if len(Vulnerability_scores_Manila) < max_length:
    Vulnerability_scores_Manila.extend([Vulnerability_scores_Manila[-1]] * (max_length - len(Vulnerability_scores_Manila)))

generations = list(range(1, max_length + 1))

# Create a DataFrame for seaborn
import pandas as pd
data_Manila = pd.DataFrame({
    'Generation': generations,
    'Vulnerability Score': Vulnerability_scores_Manila,
    'Location': 'Manila'
})


# Plotting the graph
sns.lineplot(data=data_Manila, x='Generation', y='Vulnerability Score', hue='Location')
plt.xlabel('Generation')
plt.ylabel('Vulnerability Score')
plt.title('Vulnerability vs Generations for Each City')
plt.legend(title='Location')
plt.show()

In [ ]:
# Extract max_score and generations
Vulnerability_scores_Manila = [item[1]['Target Failure Score'] for item in max_score_list_Manila]

max_length = len(Vulnerability_scores_Manila)

if len(Vulnerability_scores_Manila) < max_length:
    Vulnerability_scores_Manila.extend([Vulnerability_scores_Manila[-1]] * (max_length - len(Vulnerability_scores_Manila)))

generations = list(range(1, max_length + 1))

# Create a DataFrame for seaborn
import pandas as pd
data_Manila = pd.DataFrame({
    'Generation': generations,
    'Vulnerability Score': Vulnerability_scores_Manila,
    'Location': 'Manila'
})


# Plotting the graph
sns.lineplot(data=data_Manila, x='Generation', y='Vulnerability Score', hue='Location')
plt.xlabel('Generation')
plt.ylabel('Vulnerability Score')
plt.title('Vulnerability vs Generations for Each City')
plt.legend(title='Location')
plt.show()

In [ ]:
# Extract max_score and generations
Vulnerability_scores_Manila = [item[1]['Target Failure Score'] for item in max_score_list_Manila]
Vulnerability_scores_Makati = [item[1]['Target Failure Score']for item in max_score_list_Makati]
Vulnerability_scores_Mandaluyong = [item[1]['Target Failure Score'] for item in max_score_list_Mandaluyong]

max_length = max(len(Vulnerability_scores_Manila), len(Vulnerability_scores_Makati), len(Vulnerability_scores_Mandaluyong))

if len(Vulnerability_scores_Manila) < max_length:
    Vulnerability_scores_Manila.extend([Vulnerability_scores_Manila[-1]] * (max_length - len(Vulnerability_scores_Manila)))
if len(Vulnerability_scores_Makati) < max_length:
    Vulnerability_scores_Makati.extend([Vulnerability_scores_Makati[-1]] * (max_length - len(Vulnerability_scores_Makati)))
if len(Vulnerability_scores_Mandaluyong) < max_length:
    Vulnerability_scores_Mandaluyong.extend([Vulnerability_scores_Mandaluyong[-1]] * (max_length - len(Vulnerability_scores_Mandaluyong)))

generations = list(range(1, max_length + 1))

# Create a DataFrame for seaborn
import pandas as pd
data_Manila = pd.DataFrame({
    'Generation': generations,
    'Vulnerability Score': Vulnerability_scores_Manila,
    'Location': 'Manila'
})

data_Makati = pd.DataFrame({
    'Generation': generations,
    'Vulnerability Score': Vulnerability_scores_Makati,
    'Location': 'Makati'
})

data_Mandaluyong = pd.DataFrame({
    'Generation': generations,
    'Vulnerability Score': Vulnerability_scores_Mandaluyong,
    'Location': 'Mandaluyong'
})

# Combine all DataFrames into one
data = pd.concat([data_Manila, data_Makati, data_Mandaluyong])

# Plotting the graph
sns.lineplot(data=data, x='Generation', y='Vulnerability Score', hue='Location')
plt.xlabel('Generation')
plt.ylabel('Vulnerability Score')
plt.title('Vulnerability vs Generations for Each City')
plt.legend(title='Location')
plt.show()

#### Robustness VS Generations

In [ ]:
# Extract max_score and generations
Robustness_scores_Manila = [item[1]['Random Failure Score'] for item in max_score_list_Manila]
Robustness_scores_Makati = [item[1]['Random Failure Score']for item in max_score_list_Makati]
Robustness_scores_Mandaluyong = [item[1]['Random Failure Score'] for item in max_score_list_Mandaluyong]

max_length = max(len(Robustness_scores_Manila), len(Robustness_scores_Makati), len(Robustness_scores_Mandaluyong))

if len(Robustness_scores_Manila) < max_length:
    Robustness_scores_Manila.extend([Robustness_scores_Manila[-1]] * (max_length - len(Robustness_scores_Manila)))
if len(Robustness_scores_Makati) < max_length:
    Robustness_scores_Makati.extend([Robustness_scores_Makati[-1]] * (max_length - len(Robustness_scores_Makati)))
if len(Robustness_scores_Mandaluyong) < max_length:
    Robustness_scores_Mandaluyong.extend([Robustness_scores_Mandaluyong[-1]] * (max_length - len(Robustness_scores_Mandaluyong)))

generations = list(range(1, max_length + 1))

# Create a DataFrame for seaborn
import pandas as pd
data_Manila = pd.DataFrame({
    'Generation': generations,
    'Robustness Score': Robustness_scores_Manila,
    'Location': 'Manila'
})

data_Makati = pd.DataFrame({
    'Generation': generations,
    'Robustness Score': Robustness_scores_Makati,
    'Location': 'Makati'
})

data_Mandaluyong = pd.DataFrame({
    'Generation': generations,
    'Robustness Score': Robustness_scores_Mandaluyong,
    'Location': 'Mandaluyong'
})

# Combine all DataFrames into one
data = pd.concat([data_Manila, data_Makati, data_Mandaluyong])

# Plotting the graph
sns.lineplot(data=data, x='Generation', y='Robustness Score', hue='Location')
plt.xlabel('Generation')
plt.ylabel('Robustness Score')
plt.title('Robustness vs Generations for Each City')
plt.legend(title='Location')
plt.show()

### Network Metrics Result

In [ ]:
print("Number of Generations in the Genetic Algorithm")
print(f"Makati: {len(max_score_list_Makati)}")
print(f"Manila: {len(max_score_list_Manila)}")
print(f"Mandaluyong: {len(max_score_list_Mandaluyong)}")

In [ ]:
def print_city_statistics(network, avg_betweenness, avg_closeness, longest_route, long_stops, shortest_route, short_stops, avg_route_length, avg_degree, avg_stops):
    components = network.fitness_score_components
    print(f"Weighted Connectivity: {components['Connectivity']:.4f}")
    print(f"- Transpo Stop Ratio: {components['Transpo Stop Ratio']:.4f}")
    print(f"- Num Intersections: {components['Num Intersections']}")
    print(f"- Average Transpo Stop Degree: {components['Average Transpo Degree']:.4f}")
    print(f"Weighted Network Coverage: {components['Network Coverage']:.4f}")
    print(f"Weighted Random Failure Score: {components['Random Failure Score']:.4f}")
    print(f"Weighted Target Failure Score: {components['Target Failure Score']:.4f}")
    print(f"Fitness Score: {network.fitness_score:.4f}")
    print(f"Total Stops: {len(network.graph.nodes(data=True))}")
    print(f"Shortest Route: {shortest_route:.4f}")
    print(f"Longest Route: {longest_route:.4f}")
    print(f"Average Route Length: {avg_route_length:.4f}")
    print(f"Average Degree: {avg_degree:.4f}")
    print(f"Average Number of Stops per Route: {avg_stops:.4f}")
    print(f"Diameter: {get_network_diameter(network.graph):.4f}")
    print(f"Max Betweenness Centrality: {avg_betweenness:.4f}")
    print(f"Max Closeness Centrality: {avg_closeness:.4f}")
    print("")

In [ ]:
# Display of Fitness Scores and Network Metrics

# MANILA
_, betweenness_centrality_Manila, closeness_centrality_Manila = get_node_centrality(population_Manila[0].graph)
max_betweenness_centrality_Manila = max(betweenness_centrality_Manila.values())
max_closeness_centrality_Manila = max(closeness_centrality_Manila.values())
longest_route_Manila, long_route_stops_Manila = get_longest_route(population_Manila[0].routes)
shortest_route_Manila, short_route_stops_Manila = get_shortest_route(population_Manila[0].routes)
avg_route_length_Manila = get_average_route_length(population_Manila[0].routes)
avg_degree_Manila = average_degree(population_Manila[0].graph)
avg_number_stops_Manila = average_number_of_stops_per_route(population_Manila[0].routes)

print("Optimized Manila")
network = population_Manila[0]
print_city_statistics(
    network,
    max_betweenness_centrality_Manila,
    max_closeness_centrality_Manila,
    longest_route_Manila.length,
    long_route_stops_Manila,
    shortest_route_Manila.length,
    short_route_stops_Manila,
    avg_route_length_Manila,
    avg_degree_Manila,
    avg_number_stops_Manila
)


# MAKATI
_, betweenness_centrality_Makati, closeness_centrality_Makati = get_node_centrality(population_Makati[0].graph)
max_betweenness_centrality_Makati = max(betweenness_centrality_Makati.values())
max_closeness_centrality_Makati = max(closeness_centrality_Makati.values())
longest_route_Makati, long_route_stops_Makati = get_longest_route(population_Makati[0].routes)
shortest_route_Makati, short_route_stops_Makati = get_shortest_route(population_Makati[0].routes)
avg_route_length_Makati = get_average_route_length(population_Makati[0].routes)
avg_degree_Makati = average_degree(population_Makati[0].graph)
avg_number_stops_Makati = average_number_of_stops_per_route(population_Makati[0].routes)

print("Optimized Makati")
network = population_Makati[0]
print_city_statistics(
    network,
    max_betweenness_centrality_Makati,
    max_closeness_centrality_Makati,
    longest_route_Makati.length,
    long_route_stops_Makati,
    shortest_route_Makati.length,
    short_route_stops_Makati,
    avg_route_length_Makati,
    avg_degree_Makati,
    avg_number_stops_Makati
)


# MANDALUYONG
_, betweenness_centrality_Mandaluyong, closeness_centrality_Mandaluyong = get_node_centrality(population_Mandaluyong[0].graph)
max_betweenness_centrality_Mandaluyong = max(betweenness_centrality_Mandaluyong.values())
max_closeness_centrality_Mandaluyong = max(closeness_centrality_Mandaluyong.values())
longest_route_Mandaluyong, long_route_stops_Mandaluyong = get_longest_route(population_Mandaluyong[0].routes)
shortest_route_Mandaluyong, short_route_stops_Mandaluyong = get_shortest_route(population_Mandaluyong[0].routes)
avg_route_length_Mandaluyong = get_average_route_length(population_Mandaluyong[0].routes)
avg_degree_Mandaluyong = average_degree(population_Mandaluyong[0].graph)
avg_number_stops_Mandaluyong = average_number_of_stops_per_route(population_Mandaluyong[0].routes)

print("Optimized Mandaluyong")
network = population_Mandaluyong[0]
print_city_statistics(
    network,
    max_betweenness_centrality_Mandaluyong,
    max_closeness_centrality_Mandaluyong,
    longest_route_Mandaluyong.length,
    long_route_stops_Mandaluyong,
    shortest_route_Mandaluyong.length,
    short_route_stops_Mandaluyong,
    avg_route_length_Mandaluyong,
    avg_degree_Mandaluyong,
    avg_number_stops_Mandaluyong
)


In [ ]:
# Breakdown -- UNWEIGHTED FITNESS SCORE COMPONENTS
weight_random_failure = 0.1
weight_targeted_failure = 0.1
weight_connectivity = 0.2
weight_network_coverage = 0.6

# MANILA
print("Manila GA Results - Unweighted Fitness Scores Components")
network = population_Manila[0]
components = network.fitness_score_components
print(f"Connectivity: {components['Connectivity'] / weight_connectivity:.4f}")
print(f"- Transpo Stop Ratio: {components['Transpo Stop Ratio']:.4f}")
print(f"- Num Intersections: {components['Num Intersections']}")
print(f"- Average Transpo Stop Degree: {components['Average Transpo Degree']}")
print(f"Network Coverage: {components['Network Coverage'] / weight_network_coverage:.4f}")
print(f"Random Failure Score: {components['Random Failure Score'] / weight_random_failure:.4f}")
print(f"Target Failure Score: {components['Target Failure Score'] / weight_targeted_failure:.4f}")
print()

In [ ]:
# FIRST GENERATION BEST

# Display of Fitness Scores and Network Metrics

# MANILA
_, betweenness_centrality_Manila, closeness_centrality_Manila = get_node_centrality(Manila_generation0.graph)
max_betweenness_centrality_Manila = max(betweenness_centrality_Manila.values())
max_closeness_centrality_Manila = max(closeness_centrality_Manila.values())
longest_route_Manila, long_route_stops_Manila = get_longest_route(Manila_generation0.routes)
shortest_route_Manila, short_route_stops_Manila = get_shortest_route(Manila_generation0.routes)
avg_route_length_Manila = get_average_route_length(Manila_generation0.routes)
avg_degree_Manila = average_degree(Manila_generation0.graph)
avg_number_stops_Manila = average_number_of_stops_per_route(Manila_generation0.routes)

print("Generation 0 Manila")
network = Manila_generation0
print_city_statistics(
    network,
    max_betweenness_centrality_Manila,
    max_closeness_centrality_Manila,
    longest_route_Manila.length,
    long_route_stops_Manila,
    shortest_route_Manila.length,
    short_route_stops_Manila,
    avg_route_length_Manila,
    avg_degree_Manila,
    avg_number_stops_Manila
)


# MAKATI
_, betweenness_centrality_Makati, closeness_centrality_Makati = get_node_centrality(Makati_generation0.graph)
max_betweenness_centrality_Makati = max(betweenness_centrality_Makati.values())
max_closeness_centrality_Makati = max(closeness_centrality_Makati.values())
longest_route_Makati, long_route_stops_Makati = get_longest_route(Makati_generation0.routes)
shortest_route_Makati, short_route_stops_Makati = get_shortest_route(Makati_generation0.routes)
avg_route_length_Makati = get_average_route_length(Makati_generation0.routes)
avg_degree_Makati = average_degree(Makati_generation0.graph)
avg_number_stops_Makati = average_number_of_stops_per_route(Makati_generation0.routes)

print("Generation 0 Makati")
network = Makati_generation0
print_city_statistics(
    network,
    max_betweenness_centrality_Makati,
    max_closeness_centrality_Makati,
    longest_route_Makati.length,
    long_route_stops_Makati,
    shortest_route_Makati.length,
    short_route_stops_Makati,
    avg_route_length_Makati,
    avg_degree_Makati,
    avg_number_stops_Makati
)


# MANDALUYONG
_, betweenness_centrality_Mandaluyong, closeness_centrality_Mandaluyong = get_node_centrality(Mandaluyong_generation0.graph)
max_betweenness_centrality_Mandaluyong = max(betweenness_centrality_Mandaluyong.values())
max_closeness_centrality_Mandaluyong = max(closeness_centrality_Mandaluyong.values())
longest_route_Mandaluyong, long_route_stops_Mandaluyong = get_longest_route(Mandaluyong_generation0.routes)
shortest_route_Mandaluyong, short_route_stops_Mandaluyong = get_shortest_route(Mandaluyong_generation0.routes)
avg_route_length_Mandaluyong = get_average_route_length(Mandaluyong_generation0.routes)
avg_degree_Mandaluyong = average_degree(Mandaluyong_generation0.graph)
avg_number_stops_Mandaluyong = average_number_of_stops_per_route(Mandaluyong_generation0.routes)

print("Generation 0 Makati")
network = Mandaluyong_generation0
print_city_statistics(
    network,
    max_betweenness_centrality_Mandaluyong,
    max_closeness_centrality_Mandaluyong,
    longest_route_Mandaluyong.length,
    long_route_stops_Mandaluyong,
    shortest_route_Mandaluyong.length,
    short_route_stops_Mandaluyong,
    avg_route_length_Mandaluyong,
    avg_degree_Mandaluyong,
    avg_number_stops_Mandaluyong
)


### Route Breakdown

In [ ]:
import osmnx as ox
import folium
from shapely.geometry import mapping

# Fetch the boundary of Manila
manila_boundary = ox.geocode_to_gdf('Mandaluyong, Philippines')

# Initialize a Folium map centered around Manila
m = folium.Map(location=[14.599512, 120.984222], zoom_start=12)

# Add the Manila boundary to the map
folium.GeoJson(manila_boundary).add_to(m)

# Save or display the map
m.save('manila_boundary_map.html')